# Code for cleaning data

In [1]:

import pandas as pd
import matplotlib.pyplot as plt
# Dependencies
import requests
from pprint import pprint
import numpy as np
import gmaps
import json
import os
from scipy.stats import linregress
from datetime import datetime
import time

# Google developer API key
from config import g_key
from collections import Counter

## Data Pull and Clean

In [2]:
#retreives weather data from a .csv called "conventional_weather_stations_inmet_brazil_1961_2019" and delimits it by a ";"
rain_data = pd.read_csv("csv_files/conventional_weather_stations_inmet_brazil_1961_2019.csv", sep=";")

#removes column "Unnamed: 19" from rain_data dataframe
del rain_data["Unnamed: 19"]

#locates the "Data" ("Date") column and coverts it into a standard format
rain_data["Data"]=pd.to_datetime(rain_data['Data'])

#splits "Data" column into 3 new columns by Day, Month, and Year
rain_data['Day'] = rain_data['Data'].dt.day
rain_data['Month'] = rain_data['Data'].dt.month
rain_data['Year'] = rain_data['Data'].dt.year
rain_data=rain_data.loc[rain_data["Year"] >1987].reset_index()

#Renames all columns in data frame from their original Portuguese to English
rain_data = rain_data.rename(columns={"Estacao": "Station", "Data": "Date", "Hora":"Time","Precipitacao":"Precipitation","UmidadeRelativa":"Relative Humidity","PressaoAtmEstacao":"Weather Presure","PressaoAtmMar":"Sea Pressure","TempBulboSeco":"DryBulb Temp","TempBulboUmido":"WetBulb Temp","TempMaxima":"Maximum Temp","TempMinima":"Minimal Temp","DirecaoVento":"Wind Ventilation","Insolacao":"Isolation","Nebulosidade":"Cloudiness","Evaporacao Piche":"Tar Evaporation","Temp Comp Media": "Average Temp","Umidade Relativa Media":"Average Humidity","Velocidade do Vento Media":"Average Wind Speed"})

#isolates precipitation data that is greater than or equal to 0.0 mm
rain_df = rain_data.loc[rain_data["Precipitation"]>=0.0]

#Imports station_and_state.csv into a data frame and drops "Unnamed: 0" column
station_df = pd.read_csv("useable_csv/station_and_state.csv")
station_df =station_df.drop(["Unnamed: 0"], axis=1)

#drops columns that will not be used in data analysis
rain_df=rain_df.drop(["Date","DryBulb Temp", "Weather Presure", "Sea Pressure","Wind Ventilation","WetBulb Temp", "Maximum Temp", "Minimal Temp", "Relative Humidity", "VelocidadeVento", "Isolation", "Cloudiness","Tar Evaporation","Average Temp", "Average Humidity","Average Wind Speed","Day","Month"],axis =1)

#merges station location data with rainfall data
rs = rain_df.merge(station_df,on="Station")

#locates only states where deforestation is occuring. 
states=['AC','AP',"AM","MA","MT","PA","RO","RR","TO"]
rain_state=rs.loc[rs["State"].isin(states)]
#rain_state.to_csv("..csv_files/Rain_by_state.csv")
rain_state

,index,Station,Time,Precipitation,Year,Station Name,State
0,18988,82024,1200,0.0,1988,BOA VISTA,RR
1,18991,82024,1200,0.0,1988,BOA VISTA,RR
2,18994,82024,1200,0.0,1988,BOA VISTA,RR
3,18997,82024,1200,0.0,1988,BOA VISTA,RR
4,19000,82024,1200,0.0,1988,BOA VISTA,RR
...,...,...,...,...,...,...,...
2532791,11950956,83364,1200,0.0,2019,PADRE RICARDO REMETTER,MT
2532792,11950959,83364,1200,2.5,2019,PADRE RICARDO REMETTER,MT
2532793,11950962,83364,1200,1.4,2019,PADRE RICARDO REMETTER,MT
2532794,11950965,83364,1200,23.6,2019,PADRE RICARDO REMETTER,MT


## Store Clean Data to Useable CSV

In [10]:
#finds all weather stations associated with each weather station's data for precipitation and calculates the yearly average of total rainfall for each state
results=rain_analysis.reset_index().groupby(["Year","State"]).mean()
trys=results.reset_index()
trys.to_csv('useable_csv/total_rainfall_by_year_by_state.csv')
trys

,Year,State,Precipitation
0,1988,AC,1805.700000
1,1988,AM,2763.875000
2,1988,AP,2591.000000
3,1988,MA,1876.308333
4,1988,MT,1397.666667
...,...,...,...
245,2019,MA,1542.583333
246,2019,MT,1452.685714
247,2019,PA,2402.520000
248,2019,RR,2196.150000


### Experimental Code

In [3]:

#all_years=rain_state["Year"].unique().tolist()
#all_years=sorted(all_years)
#all_state_stations=[]


#for i in all_years:
    #group dataframe by Year and State and calculates the average rainfall per state per year
    #rain_dict=dict(Counter(rain_state.groupby(["Year","State","Station Name","State"]).head().set_index("Year").loc[i].reset_index().groupby(["Year","State","Station Name","State"]).count().index.get_level_values(3)))
    #all_state_stations.append(rain_dict.copy())
    #yr_st_count=dict(Counter(rain_state[1988].groupby(["State","Station Name","State"]).count().index.get_level_values(2).tolist()))
    #yr_st_count

#d1=dict(zip(all_years,all_state_stations))



#rain_analysis = rain_state.groupby(["Year","Station Name","State"]).agg('sum').drop(["index","Time","Station"],axis=1)
#rain_analysis

#results=rain_analysis.groupby(["Year","State"]).mean().reset_index().drop(["index"],axis=1)
#results=results.drop(results.columns[2],axis=1)
#results.to_csv('useable_csv/total_rainfall_by_year_by_state.csv')
#results

#test_r=rain_analysis.set_index("Year")

#result=test_r[1988,"Precipitation"]/d1[test_r.index[0]][test_r.iloc[0,]]